In [1]:
import pandas as pd
test = pd.read_parquet('EUR_USD_5min_1H_indic_not_norm_unbiased.parquet')
test

,open,high,low,close,sma_20,sma_50,rsi,macd,macd_signal,macd_hist,bb_upper,bb_middle,bb_lower,bb_bandwidth,bb_percent,atr,plus_di,minus_di,adx


In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager
from visualization.chart_manager import ChartManager
from data_management.preprocessor import DataPreprocessor

processor = DataPreprocessor()
dataset_manager = DatasetManager()
chart_manager = ChartManager()

eur_1min_spirce = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/1min/EUR_USD.parquet'
df = pd.read_parquet(eur_1min_spirce)
indicator_manager = IndicatorManager()
resampled_df = df.resample('5min').agg({
                    'open': 'first',
                    'high': 'max',
                    'low': 'min',
                    'close': 'last',
                }).dropna()
df_with_indicators = indicator_manager.calculate_indicators(resampled_df, indicator_timeframe='1H')
# df_with_indicators.dropna(inplace=True)

# print(df_with_indicators)
df_norm = processor.normalize_simple(df=df_with_indicators)
df_norm.to_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_15T_norm.parquet')
df_norm

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager



all_tickers = [
            'GBP_CHF', 'GBP_JPY', 'EUR_CHF', 'EUR_JPY', 'USD_CHF',
            'EUR_CAD', 'EUR_USD', 'GBP_USD', 'EUR_GBP', 'USD_JPY',
            'USD_CAD', 'AUD_USD', 'CHF_JPY', 'AUD_JPY', 'NZD_USD',
            'NZD_JPY', 'XAU_USD', 'XAG_USD'
        ]

for ticker in all_tickers:
    train_set = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/{ticker}_5T_indics_1H_norm.parquet'
    df = pd.read_parquet(train_set)
    dataset_manager = DatasetManager()
    train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

In [6]:
import pandas as pd
import os
pair = 'EUR_USD'
base_storage_path="/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm/to_test_deploy"
currency = f"{pair}_5T_indics_1H_not_norm.parquet"
path = os.path.join(base_storage_path, currency)
checking = pd.read_parquet(path)
checking

,open,high,low,close,sma_20,sma_50,rsi,macd,macd_signal,macd_hist,...,bb_bandwidth,bb_percent,atr,plus_di,minus_di,adx,senkou_span_a,senkou_span_b,tenkan_sen,kijun_sen
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-09-22 20:00:00+00:00,1.11600,1.11610,1.11600,1.11610,1.108475,1.102776,60.483592,0.002846,0.002346,0.000500,...,1.525012,92.740407,0.005111,25.821356,11.596413,27.702519,1.100825,1.09185,1.10960,1.110050
2024-09-22 20:05:00+00:00,1.11610,1.11610,1.11600,1.11600,1.108475,1.102776,60.483592,0.002846,0.002346,0.000500,...,1.525012,92.740407,0.005111,25.821356,11.596413,27.702519,1.100825,1.09185,1.10960,1.110050
2024-09-22 20:10:00+00:00,1.11600,1.11600,1.11600,1.11600,1.108475,1.102776,60.483592,0.002846,0.002346,0.000500,...,1.525012,92.740407,0.005111,25.821356,11.596413,27.702519,1.100825,1.09185,1.10960,1.110050
2024-09-22 20:15:00+00:00,1.11600,1.11620,1.11600,1.11620,1.108475,1.102776,60.483592,0.002846,0.002346,0.000500,...,1.525012,92.740407,0.005111,25.821356,11.596413,27.702519,1.100825,1.09185,1.10960,1.110050
2024-09-22 20:20:00+00:00,1.11620,1.11630,1.11620,1.11630,1.108475,1.102776,60.483592,0.002846,0.002346,0.000500,...,1.525012,92.740407,0.005111,25.821356,11.596413,27.702519,1.100825,1.09185,1.10960,1.110050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-02 20:55:00+00:00,1.05040,1.05040,1.04996,1.05001,1.054796,1.073527,39.482812,-0.006569,-0.007944,0.001375,...,2.641906,32.715948,0.007814,13.061007,29.874290,41.621309,1.086770,1.09860,1.04646,1.063465
2024-12-02 21:00:00+00:00,1.05000,1.05004,1.04970,1.04976,1.054796,1.073527,39.482812,-0.006569,-0.007944,0.001375,...,2.641906,32.715948,0.007814,13.061007,29.874290,41.621309,1.086770,1.09860,1.04646,1.063465
2024-12-02 21:05:00+00:00,1.04977,1.05006,1.04976,1.05004,1.054796,1.073527,39.482812,-0.006569,-0.007944,0.001375,...,2.641906,32.715948,0.007814,13.061007,29.874290,41.621309,1.086770,1.09860,1.04646,1.063465


In [3]:
import pandas as pd
flo = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm/AUD_JPY_5T_indics_1H_not_norm.parquet'
# checking2 = pd.read_parquet(f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/EUR_USD_5T_indics_1H_norm.parquet')
checking2 = pd.read_parquet(flo)
checking2

,open,high,low,close,sma_20,sma_50,rsi,macd,macd_signal,macd_hist,...,bb_bandwidth,bb_percent,atr,plus_di,minus_di,adx,senkou_span_a,senkou_span_b,tenkan_sen,kijun_sen
timestamp,,,,,,,,,,,,,,,,,,,,,
2001-01-08 06:00:00+00:00,66.280,66.280,66.280,66.280,66.37300,65.6660,49.731639,0.143734,0.230073,-0.086339,...,0.82436,14.726479,0.170494,24.780306,19.872353,44.009203,65.4650,64.8650,66.415,66.4050
2001-01-08 06:05:00+00:00,66.280,66.280,66.270,66.280,66.37300,65.6660,49.731639,0.143734,0.230073,-0.086339,...,0.82436,14.726479,0.170494,24.780306,19.872353,44.009203,65.4650,64.8650,66.415,66.4050
2001-01-08 06:10:00+00:00,66.280,66.290,66.280,66.280,66.37300,65.6660,49.731639,0.143734,0.230073,-0.086339,...,0.82436,14.726479,0.170494,24.780306,19.872353,44.009203,65.4650,64.8650,66.415,66.4050
2001-01-08 06:15:00+00:00,66.280,66.280,66.240,66.240,66.37300,65.6660,49.731639,0.143734,0.230073,-0.086339,...,0.82436,14.726479,0.170494,24.780306,19.872353,44.009203,65.4650,64.8650,66.415,66.4050
2001-01-08 06:20:00+00:00,66.230,66.240,66.230,66.240,66.37300,65.6660,49.731639,0.143734,0.230073,-0.086339,...,0.82436,14.726479,0.170494,24.780306,19.872353,44.009203,65.4650,64.8650,66.415,66.4050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-28 10:00:00+00:00,98.687,98.718,98.668,98.674,98.30365,98.6614,57.858416,0.000290,-0.079396,0.079686,...,0.99314,88.241601,0.196347,24.335781,20.408517,26.559452,98.9665,99.5245,98.436,98.2255
2024-11-28 10:05:00+00:00,98.674,98.698,98.650,98.690,98.30365,98.6614,57.858416,0.000290,-0.079396,0.079686,...,0.99314,88.241601,0.196347,24.335781,20.408517,26.559452,98.9665,99.5245,98.436,98.2255
2024-11-28 10:10:00+00:00,98.690,98.727,98.674,98.698,98.30365,98.6614,57.858416,0.000290,-0.079396,0.079686,...,0.99314,88.241601,0.196347,24.335781,20.408517,26.559452,98.9665,99.5245,98.436,98.2255


BEST RESULTS SO FAR

In [ ]:
import os
import sys

import pandas as pd
import numpy as np

from datetime import datetime, timedelta
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager
from visualization.chart_manager import ChartManager
from data_management.preprocessor import DataPreprocessor
processor = DataPreprocessor()
dataset_manager = DatasetManager()
chart_manager = ChartManager()
all_tickers = [
            'GBP_CHF', 'GBP_JPY', 'EUR_CHF', 'EUR_JPY', 'USD_CHF',
            'EUR_CAD', 'EUR_USD', 'GBP_USD', 'EUR_GBP', 'USD_JPY',
            'USD_CAD', 'AUD_USD', 'CHF_JPY', 'AUD_JPY', 'NZD_USD',
            'NZD_JPY', 'XAU_USD', 'XAG_USD'
        ]

for ticker in all_tickers:
    source_df = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/1min/{ticker}.parquet'
    df = pd.read_parquet(source_df)
    indicator_manager = IndicatorManager()
    resampled_df = df.resample('5min').agg({
                        'open': 'first',
                        'high': 'max',
                        'low': 'min',
                        'close': 'last',
                    }).dropna()
    df_with_indicators = indicator_manager.calculate_indicators(resampled_df, indicator_timeframe='1H')
    # df_with_indicators.dropna(inplace=True)

    # print(df_with_indicators)
    # df_norm = processor.normalize_simple(df=df_with_indicators)
    # df_norm.to_parquet(f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/{ticker}_5T_indics_1H_not_norm.parquet')
    df_with_indicators.to_parquet(f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes_not_norm/{ticker}_5T_indics_1H_not_norm.parquet')
    

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager



all_tickers = [
            'GBP_CHF', 'GBP_JPY', 'EUR_CHF', 'EUR_JPY', 'USD_CHF',
            'EUR_CAD', 'EUR_USD', 'GBP_USD', 'EUR_GBP', 'USD_JPY',
            'USD_CAD', 'AUD_USD', 'CHF_JPY', 'AUD_JPY', 'NZD_USD',
            'NZD_JPY', 'XAU_USD', 'XAG_USD'
        ]

for ticker in all_tickers:
    train_set = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/{ticker}_5T_indics_1H_norm.parquet'
    df = pd.read_parquet(train_set)
    dataset_manager = DatasetManager()
    train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)

    saving_path = f'/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/best_dataframes/models_and_vecs/'
    os.makedirs(saving_path, exist_ok=True)

    def make_train_env():
        env = ForexTradingEnv(
            df=train_df,
            pair=ticker,

        )
        env = Monitor(env)
        env = DummyVecEnv([lambda: env])
        env = VecNormalize(env, norm_obs=True, norm_reward=True)
        return env

    def make_eval_env():
        env = ForexTradingEnv(

            df=val_df,
            pair=ticker,
            # resample_interval='1h'
        )
        env = Monitor(env)
        env = DummyVecEnv([lambda: env])
        env = VecNormalize(env, norm_obs=True, norm_reward=False)
        env.training = False
        return env

    train_env = make_train_env()
    eval_env = make_eval_env()
    eval_callback = EvalCallback(
        eval_env,
        best_model_save_path=saving_path,
        log_path=saving_path,
        eval_freq=50_000,  # Adjust as needed
        n_eval_episodes=5,
        deterministic=True,
        render=False
    )

    model = PPO(
        'MlpPolicy',
        train_env,
        verbose=0,
        tensorboard_log=f'{saving_path}{ticker}_tensorboard/',
    )

    model.learn(
        total_timesteps=3_000_000,  # Adjust as needed
        callback=eval_callback
    )

    model.save(f'{saving_path}{ticker}_best_model.zip')
    train_env.save(f'{saving_path}{ticker}_vec_normalize.pkl')

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager

train_set = '/Volumes/ssd_fat2/ai6_trading_bot/datasets/5min/EUR_USD_5T_indics_15T_norm.parquet'

df = pd.read_parquet(train_set)
dataset_manager = DatasetManager()
train_df, val_df, test_df = dataset_manager.split_dataset(df, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)



saving_path = f'./logs/29nov/norm_15min_1h_indic/'
os.makedirs(saving_path, exist_ok=True)

def make_train_env():
    env = ForexTradingEnv(
        df=train_df,
        pair='EUR_USD',

    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=True)
    return env

def make_eval_env():
    env = ForexTradingEnv(

        df=val_df,
        pair='EUR_USD',
        # resample_interval='1h'
    )
    env = Monitor(env)
    env = DummyVecEnv([lambda: env])
    env = VecNormalize(env, norm_obs=True, norm_reward=False)
    env.training = False
    return env

train_env = make_train_env()
eval_env = make_eval_env()
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path=saving_path,
    log_path=saving_path,
    eval_freq=50_000,  # Adjust as needed
    n_eval_episodes=5,
    deterministic=True,
    render=False
)

model = PPO(
    'MlpPolicy',
    train_env,
    verbose=0,
    tensorboard_log=f'{saving_path}tensorboard/',
)

model.learn(
    total_timesteps=3_000_000,  # Adjust as needed
    callback=eval_callback
)

model.save(f'{saving_path}best_model.zip')
train_env.save(f'{saving_path}vec_normalize.pkl')

In [ ]:
import os
import sys

import pandas as pd

from datetime import datetime, timedelta
from pathlib import Path


# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize, DummyVecEnv
from trading.environments.forex_env2_flat import ForexTradingEnv
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
from data_management.dataset_manager import DatasetManager
from data_management.indicator_manager import IndicatorManager

min5_not_norm = pd.read_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/EUR_USD.parquet')
from visualization.chart_manager import ChartManager
chart_manager = ChartManager()
start_time = pd.Timestamp('2024-01-01').tz_localize('UTC')
end_time = pd.Timestamp('2024-10-31').tz_localize('UTC')
chart_manager.chart(min5_not_norm, start_time, end_time)

In [ ]:
min5_not_norm = pd.read_parquet('/Volumes/ssd_fat2/ai6_trading_bot/datasets/1h/EUR_USD.parquet')
min5_not_norm